In [ ]:
pip install gymnasium

In [ ]:
pip install swig

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 29.0 MB/s eta 0:00:00


In [ ]:
pip install "gymnasium[box2d]"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.4/374.4 kB 7.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for box2d-py: filename=box2d_py-2.3.5-cp311-cp311-linux_x86_64.whl size=2379370 sha256=ce1cfbc90be1f6dd955a8094c5d21e35e88881d50b1b72f65a9f30029b174f08
  Stored in directory: /root/.cache/pip/wheels/ab/f1/0c/d56f4a2bdd12bae0a0693ec33f2f0daadb5eb9753c78fa5308
Successfully built box2d-py


All of the imports used in this project:

*   gymnasium: used to get an environment for the project.
*   numpy: used for numerical operations.
*   tourch: PyTorhc is used for building and also training the nueral networks.
*   collections: used for the replay buffer



In [ ]:
import gymnasium as gym

In [ ]:
import os
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.autograd as autograd
from torch.autograd import Variable
from collections import deque, namedtuple

Lunar Lander Environment, this setup os the default one given in the Gymnasium Documentaion.

In [ ]:
env = gym.make("LunarLander-v3", continuous=False, gravity=-10.0,
               enable_wind=False, wind_power=15.0, turbulence_power=1.5)

Gets the shape size of the state space, and also the number of actions

Action Space (discrete):
- 0: Do nothing
- 1: Fire left orientation engine
- 2: Fire main engine
- 3: Fire right orientation engine

In [ ]:
state_shape = env.observation_space.shape
state_size = env.observation_space.shape[0]
number_actions = env.action_space.n
print('State shape: ', state_shape)
print('State size: ', state_size)
print('Number of actions: ', number_actions)

State shape:  (8,)
State size:  8
Number of actions:  4


Hyperparameters, these define:

*   Learing rate for the optimizer (α)
*   Mini-batch size for sampling experiences
*   Discount factiors for comming rewards.(γ)
*   Replay Buffer capazity
*   Interpolation factos for soft updates of the target network.

In [ ]:
learning_rate = 5e-4
minibatch_size = 100
discount_factor = 0.99
replay_buffer_size = int(1e5)
interpolation_parameter = 1e-3

The class ANN defienes a 3-layer neural network:

*   Input layer: state_size
*   Hidden layers: 64 units each
*   Output layer: action_size (Q-values for each of the actions)

This is the Q-network, estimating:

𝑄
(
𝑠
,
𝑎
)
≈
expected return starting from state
𝑠
 and taking action
𝑎

So it outputs one Q-value per possible action.

In [ ]:
class ANN(nn.Module):
  def __init__(self, state_size, action_size, seed = 42):
    super(ANN, self).__init__()
    self.seed = torch.manual_seed(seed)
    self.fc1 = nn.Linear(state_size, 64)
    self.fc2 = nn.Linear(64,64)
    self.fc3 = nn.Linear(64, action_size)

  def forward(self, state):
    x = self.fc1(state)
    x = F.relu(x)
    x = self.fc2(x)
    x = F.relu(x)
    return self.fc3(x)

This class is the Replay Buffer that:


*   Stores tuples of experiences: (state, action. reward, next_state, done)
*   Allows random sampling for experience replay

Sampling returns: states, actions, rewards, next_states, dones



In [ ]:
class Replaybuffer(object):
  def __init__(self, capacity):
    self.capacity = capacity
    self.memory = []

  def push(self, event):
    self.memory.append(event)
    if len(self.memory) > self.capacity:
      del self.memory[0]

  def sample(self, batch_size):
    experiences = random.sample(self.memory, batch_size)

    states = torch.from_numpy(np.vstack([e[0] for e in experiences])).float()
    actions = torch.from_numpy(np.vstack([e[1] for e in experiences])).long()
    rewards = torch.from_numpy(np.vstack([e[2] for e in experiences])).float()
    next_states = torch.from_numpy(np.vstack([e[3] for e in experiences])).float()
    dones = torch.from_numpy(np.vstack([e[4] for e in experiences]).astype(np.uint8)).float()

    return states, next_states, actions, rewards, dones

The class Agent (DQN Agent) encapsulates the reinforment learning logic:

*   Initializes two networks: Local and Target

local_qnetwork: Used for choosing actions

target_qnetwork: Used for stable target calculation
*   Uses epsilon-greedy for exploration
*   Stores experiences and samples mini-batches for training
*   Performs soft updates in the target network

Step Function:
Stores the transition and learns every 4 steps if enough memory exists.

Act Function: Selects an Action to be taken
- Convert NumPy state to Torch tensor and move it to GPU/CPU.
- unsqueeze(0) adds a batch dimension, making shape (1, state_size).

Learning / Update Rule
Key equation:

𝑄
^
(
𝑠
,
𝑎
)
=
𝑟
+
𝛾
⋅
max
⁡
𝑎
′
𝑄
target
(
𝑠
′
,
𝑎
′
)
(TD Target)
Q
^
​
 (s,a)=r+γ⋅
a
′

max
​
 Q
target
​
 (s
′
 ,a
′
 )(TD Target)

 Soft Update:

𝜃
target
←
𝜏
⋅
𝜃
local
+
(
1
−
𝜏
)
⋅
𝜃
target

In [ ]:
class Agent():

  def __init__(self, state_size, action_size):
    self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    self.state_size = state_size
    self.action_size = action_size
    self.local_qnetwork = ANN(state_size, action_size).to(self.device)
    self.target_qnetwork = ANN(state_size, action_size).to(self.device)
    self.optimizer = optim.Adam(self.local_qnetwork.parameters(), lr = learning_rate)
    self.memory = Replaybuffer(replay_buffer_size)
    self.t_step = 0

  def step(self, state, action, reward, next_state, done):
    self.memory.push((state, action, reward, next_state, done))
    self.t_step = (self.t_step + 1) % 4
    if self.t_step == 0:
      if len(self.memory.memory) > minibatch_size:
        experiences = self.memory.sample(100)
        self.learn(experiences, discount_factor)

  def act(self, state, epsilon = 0.):
    state = torch.from_numpy(state).float().unsqueeze(0).to(self.device)
    self.local_qnetwork.eval()
    with torch.no_grad():
      action_values = self.local_qnetwork(state)
    self.local_qnetwork.train()
    if random.random() > epsilon:
      return np.argmax(action_values.cpu().data.numpy())
    else:
      return random.choice(np.arange(self.action_size))

  def learn(self, experiences, discount_factor):
    states, next_states, actions, rewards, dones = experiences

    states = states.to(self.device)
    next_states = next_states.to(self.device)
    actions = actions.to(self.device)
    rewards = rewards.to(self.device)
    dones = dones.to(self.device)

    next_q_targets = self.target_qnetwork(next_states).detach().max(1)[0].unsqueeze(1)
    q_targets = rewards + discount_factor * next_q_targets * (1 - dones)
    q_expected = self.local_qnetwork(states).gather(1, actions)
    loss = F.mse_loss(q_expected, q_targets)
    self.optimizer.zero_grad()
    loss.backward()
    self.optimizer.step()
    self.soft_update(self.local_qnetwork, self.target_qnetwork, interpolation_parameter)


  def soft_update(self, local_model, target_model, interpolation_parameter):
    for target_param, local_param in zip(target_model.parameters(), local_model.parameters()):
      target_param.data.copy_(interpolation_parameter * local_param.data + (1.0 - interpolation_parameter) * target_param.data)


In [ ]:
agent = Agent(state_size, number_actions)

Reset the environment

For each time step:

- Choose action with epsilon-greedy

- Perform the action → observe reward and next state

- Store experience

- Learn if ready

Decay epsilon:

ϵ=max(ϵ
min
​
 ,ϵ⋅ϵ
decay
​
 )

In [ ]:
number_episodes = 2000
maximum_number_timesteps_per_episode = 1000
epsilon_starting_value  = 1.0
epsilon_ending_value  = 0.01
epsilon_decay_value  = 0.995
epsilon = epsilon_starting_value
scores_on_100_episodes = deque(maxlen = 100)

for episode in range(1, number_episodes + 1):
  state, _ = env.reset()
  score = 0
  for t in range(maximum_number_timesteps_per_episode):
    action = agent.act(state, epsilon)
    next_state, reward, done, _, _ = env.step(action)
    agent.step(state, action, reward, next_state, done)
    state = next_state
    score += reward
    if done:
      break
  scores_on_100_episodes.append(score)
  epsilon = max(epsilon_ending_value, epsilon_decay_value * epsilon)
  print('\rEpisode {}\tAverage Score: {:.2f}'.format(episode, np.mean(scores_on_100_episodes)), end = "")
  if episode % 100 == 0:
    print('\rEpisode {}\tAverage Score: {:.2f}'.format(episode, np.mean(scores_on_100_episodes)))
  if np.mean(scores_on_100_episodes) >= 200.0:
    print('\nEnvironment had solved been solved. \tAverage Score: {:.2f}'.format(episode - 100, np.mean(scores_on_100_episodes)))
    torch.save(agent.local_qnetwork.state_dict(), 'checkpoint.pth')
    break

Episode 100	Average Score: -101.57
Episode 200	Average Score: -36.69
Episode 300	Average Score: 29.90
Episode 400	Average Score: 121.26
Episode 442	Average Score: 200.11
Environment had solved been solved. 	Average Score: 342.00


Shows video of the trainmed agent in the environment:

In [ ]:
import glob
import io
import base64
import imageio
from IPython.display import HTML, display

def show_video_of_model(agent, env_name):
    env = gym.make(env_name, render_mode='rgb_array')
    state, _ = env.reset()
    done = False
    frames = []
    while not done:
        frame = env.render()
        frames.append(frame)
        action = agent.act(state)
        state, reward, done, _, _ = env.step(action)
    env.close()
    imageio.mimsave('video.mp4', frames, fps=30)

show_video_of_model(agent, 'LunarLander-v3')

def show_video():
    mp4list = glob.glob('*.mp4')
    if len(mp4list) > 0:
        mp4 = mp4list[0]
        video = io.open(mp4, 'r+b').read()
        encoded = base64.b64encode(video)
        display(HTML(data=f'''
        <video width="640" height="480" controls>
          <source src="data:video/mp4;base64,{encoded.decode('ascii')}" type="video/mp4">
        </video>
'''))
    else:
        print("Could not find video")

show_video()